# How Two Overwrite Feature Layers on AcrGIS Online Using ArcGIS API for Python

#### This script was used to update a park Police Feature layer every hour. Everytime this feature layer is updated, this [Live - Incidents Dashboard](https://www.arcgis.com/apps/dashboards/507bf64453764a608f2323475bb52f2f) is also updated to help park police prioritize incidents throughout the Park. 

#### In this tutorial we will overwrite a feature layer that was created in an ArcGIS Pro project. In ArcGIS Pro, the layer was first created using data from a connected MSSQL database table. The layer was then shared through ArcGIS Pro to ArcGIS Online (AGO). The MSSQL table regularly updates and we want to reflect those updates on the shared feature layer. Here we will set up a script that overwrites the feature layer with new sql table updates. After your finished making the script, use windows task scheduler to regulaly run this script and update your AGO feature layer. Click and scroll down to ["Create and Schedule a Task"](https://learn.arcgis.com/en/projects/schedule-automated-near-real-time-data-updates/) for more information and Windows task scheduler.

First, lets read in nessesary libraries

Note: You may also need to download the [Overwrite Hosted Feature Services, v2.1.1](https://www.arcgis.com/home/item.html?id=d45f80eb53c748e7aa3d938a46b48836) and install the [Manage Notebook Code Dependencies (MNCD) tool Installer ](https://www.arcgis.com/home/item.html?id=46c7512604654601ab4338f9299c5414) and set up your environment.

In [1]:
import arcpy
import os, sys 
from arcgis.gis import GIS

Now, lets set up some variables for the project path, feature service/SD name in arcgis.com,
the user/password of the owner account, and the sharing options for the item you will be overwriting on AGO.
If you have your login credentials locally stored as a saved profile, you don't need to write your password in this script and can delete it.
To learn how to locally store your 
AGO credentials visit [this tutorial](https://learn.arcgis.com/en/projects/schedule-automated-near-real-time-data-updates/) and scroll down to the section "Store ArcgGIS Online Credentials Locally". 

In [2]:
prjPath = r"C:\Users\YourName\Desktop\ArcProjectFolder\YourProjectName.aprx" #fill in path to your project path. 


sd_fs_name = "The name of the feature layer to be updated"
# Can also reference your organization portal
portal = 'https://www.arcgis.com'


#login credentials
user = "Your_User_Name"

#Comment these two lines out if you have a locally stored profile and will be using it.
password = "Your Password" 
gis = GIS(portal, user, password)

#Comment this out if you don't have a locally stored profile and 
# will be keeping your user name and password in this script
gis = GIS( profile="learn_user")


# Set sharing options
shrOrg = False
shrEveryone = True
shrGroups = ""

### End setting variables

When a feature layer is first shared to AGO via ArcGIS Pro, you might notice that a Service Definition (SD) for that feature layer is created in your AGO content.
To overwrite the current feature layer with updated content, we will have to define local paths to create temporary contentent, create the new SDDraft, and stage to SD.

In [3]:
# Local paths to create temporary content
relPath = sys.path[0]
sddraft = os.path.join(relPath, "WebUpdate.sddraft")
sd = os.path.join(relPath, "WebUpdate.sd") 


# Create a new SDDraft and stage to SD
print("Creating SD file")
arcpy.env.overwriteOutput = True
prj = arcpy.mp.ArcGISProject(prjPath)
mp = prj.listMaps()[0]
arcpy.mp.CreateWebLayerSDDraft(mp, sddraft, sd_fs_name, "MY_HOSTED_SERVICES", "FEATURE_ACCESS","", True, True)
arcpy.StageService_server(sddraft, sd)

# Connect to your AGO portal
print("Connecting to {}".format(portal))

Creating SD file
Connecting to https://hrpt.maps.arcgis.com/


Now that you have created the temporary content and connected to your portal, search for the SD item on AGO, update the item, and publish the SD.

In [4]:
# Find the SD, update it, publish /w overwrite and set sharing and metadata
print("Search for original SD on portal…")
sdItem = gis.content.search("{} AND owner:{}".format(sd_fs_name, user), item_type="Service Definition")[0]
print("Found SD: {}, ID: {} n Uploading and overwriting…".format(sdItem.title, sdItem.id))
sdItem.update(data=sd)
print("Overwriting existing feature service…")
fs = sdItem.publish(overwrite=True)


if shrOrg or shrEveryone or shrGroups:
    print("Setting sharing options…")
    fs.share(org=shrOrg, everyone=shrEveryone, groups=shrGroups)

print("Finished updating: {} – ID: {}".format(fs.title, fs.id))


Search for original SD on portal…
Found SD: Live_PEP_Features, ID: bd7e45bcc39d4333b6a23608a6654a97 n Uploading and overwriting…
Overwriting existing feature service…
Setting sharing options…
Finished updating: Live_PEP_Features – ID: c39af8d4288740d9b67627f2143d28cb
